In [15]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional
from dotenv import load_dotenv
from boeing_chat_model import BoeingChatModel
import os

# This needs to be replaced with OpenAI Chat Models

In [4]:
load_dotenv()

PAT = os.getenv("PAT")

In [6]:
model = BoeingChatModel(
    udal_pat=PAT
)

In [7]:
class LLMState(TypedDict):

    question: str
    answer: str

In [16]:
from pydantic import BaseModel, Field

class LLMStatePydantic(BaseModel):

    question: str = Field(..., description="The question to ask")
    answer: Optional[str] = Field(None, description="The answer to the question")

In [17]:
def llm_qa(state: LLMStatePydantic) -> LLMStatePydantic:
    question = state.question
    answer = model.invoke(question)
    state.answer = answer.content
    return state

In [18]:
graph = StateGraph(LLMStatePydantic)

# Node
graph.add_node("llm_qa", llm_qa)

# Edges
graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

# Compile
workflow = graph.compile()

In [19]:
# Execute
initial_state = {"question": "What is Football?"}
final_state = workflow.invoke(initial_state)
final_state

{'question': 'What is Football?',
 'answer': '## What is football?\n\n"Football" is a general name for several team sports that involve advancing a ball to score points. Which sport someone means depends on where they live. The two most widely used meanings are:\n\n- Association football (commonly called “soccer” in the United States and Canada)\n  - Objective: Score by getting the ball into the opponent’s goal.\n  - Players: 11 per side on the field.\n  - Ball handling: Mostly played with the feet; only goalkeepers may handle the ball inside their penalty area.\n  - Field: Rectangular grass or artificial turf with goals at each end.\n  - Scoring: Each goal counts as one point.\n  - Worldwide: The world’s most popular sport, governed by FIFA; major events include the FIFA World Cup.\n\n- American football (commonly called “football” in the United States)\n  - Objective: Move the ball into the opponent’s end zone for a touchdown or kick field goals/extra points.\n  - Players: 11 on the 